# Intervention Results Analysis

This notebook analyzes the results of demographic circuit interventions on the ANES 2024 dataset.

**Analyses included:**
- Summary statistics for single-demographic and intersectional interventions
- Comparison visualizations (baseline vs intervention)
- Per-question analysis
- Per-demographic analysis (which demographics improve accuracy most)
- Advanced metrics (Kendall's tau, entropy, F1, etc.)

## 1. Setup & Configuration

In [ ]:
import pickle
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# ============================================================================
# CONFIGURATION - Modify these parameters
# ============================================================================

RUN_ID = "exp_001"  # Change this to your run ID
PROBE_TYPE = "ridge"  # ridge or logistic
RESULTS_DIR = Path("robust_experiment_results/intervention_results")

# Demographics to analyze
SINGLE_DEMOGRAPHICS = ['age', 'gender', 'ideology', 'race']
INTERSECTIONAL_DEMOGRAPHICS = ['age+gender+ideology+race']  # Add more if you have them

print(f"Configuration:")
print(f"  Run ID: {RUN_ID}")
print(f"  Probe Type: {PROBE_TYPE}")
print(f"  Results Directory: {RESULTS_DIR}")
print(f"  Single Demographics: {SINGLE_DEMOGRAPHICS}")
print(f"  Intersectional: {INTERSECTIONAL_DEMOGRAPHICS}")

## 2. Load Data

In [ ]:
# Load summary JSON
summary_file = RESULTS_DIR / f"intervention_summary_{RUN_ID}.json"

if summary_file.exists():
    with open(summary_file, 'r') as f:
        summary_data = json.load(f)
    print(f"✓ Loaded summary from: {summary_file}")
    print(f"  Mode: {summary_data.get('mode', 'N/A')}")
else:
    print(f"✗ Summary file not found: {summary_file}")
    summary_data = None

In [ ]:
# Load single-demographic results
single_demo_results = {}

for demographic in SINGLE_DEMOGRAPHICS:
    result_file = RESULTS_DIR / f"{demographic}_{PROBE_TYPE}_{RUN_ID}_intervention_results.pkl"
    
    if result_file.exists():
        with open(result_file, 'rb') as f:
            single_demo_results[demographic] = pickle.load(f)
        print(f"✓ Loaded {demographic}: {result_file.name}")
    else:
        print(f"✗ Not found: {demographic}")

print(f"\nLoaded {len(single_demo_results)} single-demographic results")

In [ ]:
# Load intersectional results
intersectional_results = {}

for demographics in INTERSECTIONAL_DEMOGRAPHICS:
    result_file = RESULTS_DIR / f"intersectional_{demographics}_{PROBE_TYPE}_{RUN_ID}_intervention_results.pkl"
    
    if result_file.exists():
        with open(result_file, 'rb') as f:
            intersectional_results[demographics] = pickle.load(f)
        print(f"✓ Loaded intersectional {demographics}: {result_file.name}")
    else:
        print(f"✗ Not found: intersectional {demographics}")

print(f"\nLoaded {len(intersectional_results)} intersectional results")

## 3. Summary Statistics Tables

### 3.1 Overall Metrics: Single Demographics

In [ ]:
# Extract overall metrics for single demographics
summary_rows = []

for demo, results in single_demo_results.items():
    metrics = results['overall_metrics']
    
    row = {
        'Demographic': demo.capitalize(),
        'Baseline Acc': f"{metrics['baseline_accuracy_mean']*100:.1f}%",
        'Baseline Std': f"{metrics['baseline_accuracy_std']*100:.1f}%" if metrics['baseline_accuracy_std'] else 'N/A',
        'Intervention Acc': f"{metrics['intervention_accuracy_mean']*100:.1f}%",
        'Intervention Std': f"{metrics['intervention_accuracy_std']*100:.1f}%" if metrics['intervention_accuracy_std'] else 'N/A',
        'Improvement': f"{metrics['improvement_mean']:+.1f}",
        'Improvement Std': f"{metrics['improvement_std']:.1f}" if metrics['improvement_std'] else 'N/A',
    }
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)
print("\n" + "="*80)
print("SINGLE-DEMOGRAPHIC INTERVENTION SUMMARY")
print("="*80)
display(summary_df)

### 3.2 Overall Metrics: Intersectional

In [ ]:
# Extract overall metrics for intersectional
intersect_rows = []

for demographics, results in intersectional_results.items():
    metrics = results['overall_metrics']
    
    row = {
        'Demographics': demographics,
        'Baseline Acc': f"{metrics['baseline_accuracy_mean']*100:.1f}%",
        'Baseline Std': f"{metrics['baseline_accuracy_std']*100:.1f}%" if metrics['baseline_accuracy_std'] else 'N/A',
        'Intervention Acc': f"{metrics['intervention_accuracy_mean']*100:.1f}%",
        'Intervention Std': f"{metrics['intervention_accuracy_std']*100:.1f}%" if metrics['intervention_accuracy_std'] else 'N/A',
        'Improvement': f"{metrics['improvement_mean']:+.1f}",
        'Improvement Std': f"{metrics['improvement_std']:.1f}" if metrics['improvement_std'] else 'N/A',
    }
    intersect_rows.append(row)

if intersect_rows:
    intersect_df = pd.DataFrame(intersect_rows)
    print("\n" + "="*80)
    print("INTERSECTIONAL INTERVENTION SUMMARY")
    print("="*80)
    display(intersect_df)
else:
    print("No intersectional results found")

### 3.3 Advanced Metrics Table

In [ ]:
# Advanced metrics for single demographics
advanced_rows = []

for demo, results in single_demo_results.items():
    metrics = results['overall_metrics']
    
    # Helper function to format metric with std
    def fmt(mean_key, std_key):
        mean_val = metrics.get(mean_key)
        std_val = metrics.get(std_key)
        if mean_val is None:
            return 'N/A'
        if std_val is None or std_val == 0:
            return f"{mean_val:.3f}"
        return f"{mean_val:.3f} ± {std_val:.3f}"
    
    row = {
        'Demographic': demo.capitalize(),
        "Kendall's τ (Baseline)": fmt('baseline_kendall_tau_mean', 'baseline_kendall_tau_std'),
        "Kendall's τ (Intervention)": fmt('intervention_kendall_tau_mean', 'intervention_kendall_tau_std'),
        'Entropy (Baseline)': fmt('baseline_entropy_mean', 'baseline_entropy_std'),
        'JS Divergence (Baseline)': fmt('baseline_js_divergence_mean', 'baseline_js_divergence_std'),
        'Macro F1 (Intervention)': fmt('intervention_macro_f1_mean', 'intervention_macro_f1_std'),
        "Cohen's κ (Intervention)": fmt('intervention_cohen_kappa_mean', 'intervention_cohen_kappa_std'),
    }
    advanced_rows.append(row)

advanced_df = pd.DataFrame(advanced_rows)
print("\n" + "="*80)
print("ADVANCED METRICS SUMMARY")
print("="*80)
display(advanced_df)

## 4. Comparison Visualizations

### 4.1 Baseline vs Intervention Accuracy

In [ ]:
# Prepare data for visualization
demographics = []
baseline_accs = []
intervention_accs = []

for demo, results in single_demo_results.items():
    metrics = results['overall_metrics']
    demographics.append(demo.capitalize())
    baseline_accs.append(metrics['baseline_accuracy_mean'] * 100)
    intervention_accs.append(metrics['intervention_accuracy_mean'] * 100)

# Create grouped bar chart
x = np.arange(len(demographics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, baseline_accs, width, label='Baseline', alpha=0.8)
bars2 = ax.bar(x + width/2, intervention_accs, width, label='Intervention', alpha=0.8)

ax.set_xlabel('Demographic Attribute')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Baseline vs Intervention Accuracy by Demographic')
ax.set_xticks(x)
ax.set_xticklabels(demographics)
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

### 4.2 Accuracy Improvement by Demographic

In [ ]:
# Prepare improvement data
demographics = []
improvements = []
improvement_stds = []

for demo, results in single_demo_results.items():
    metrics = results['overall_metrics']
    demographics.append(demo.capitalize())
    improvements.append(metrics['improvement_mean'])
    improvement_stds.append(metrics['improvement_std'] if metrics['improvement_std'] else 0)

# Sort by improvement
sorted_indices = np.argsort(improvements)[::-1]
demographics = [demographics[i] for i in sorted_indices]
improvements = [improvements[i] for i in sorted_indices]
improvement_stds = [improvement_stds[i] for i in sorted_indices]

# Create bar chart
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['green' if imp > 0 else 'red' for imp in improvements]
bars = ax.bar(demographics, improvements, yerr=improvement_stds, 
              color=colors, alpha=0.7, capsize=5)

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.set_xlabel('Demographic Attribute')
ax.set_ylabel('Accuracy Improvement (percentage points)')
ax.set_title('Accuracy Improvement by Demographic (Intervention - Baseline)')
ax.grid(axis='y', alpha=0.3)

# Add value labels
for i, (bar, imp, std) in enumerate(zip(bars, improvements, improvement_stds)):
    height = bar.get_height()
    label = f'{imp:+.1f}' if std == 0 else f'{imp:+.1f} ± {std:.1f}'
    ax.text(bar.get_x() + bar.get_width()/2., height,
            label,
            ha='center', va='bottom' if height > 0 else 'top', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\nBest improvement: {demographics[0]} ({improvements[0]:+.1f} points)")
print(f"Worst improvement: {demographics[-1]} ({improvements[-1]:+.1f} points)")

### 4.3 Kendall's Tau Comparison

In [ ]:
# Prepare Kendall's tau data
demographics = []
baseline_taus = []
intervention_taus = []

for demo, results in single_demo_results.items():
    metrics = results['overall_metrics']
    baseline_tau = metrics.get('baseline_kendall_tau_mean')
    intervention_tau = metrics.get('intervention_kendall_tau_mean')
    
    if baseline_tau is not None and intervention_tau is not None:
        demographics.append(demo.capitalize())
        baseline_taus.append(baseline_tau)
        intervention_taus.append(intervention_tau)

if demographics:
    x = np.arange(len(demographics))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 6))
    bars1 = ax.bar(x - width/2, baseline_taus, width, label='Baseline', alpha=0.8)
    bars2 = ax.bar(x + width/2, intervention_taus, width, label='Intervention', alpha=0.8)

    ax.set_xlabel('Demographic Attribute')
    ax.set_ylabel("Kendall's Tau")
    ax.set_title("Kendall's Tau: Baseline vs Intervention")
    ax.set_xticks(x)
    ax.set_xticklabels(demographics)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()
else:
    print("No Kendall's tau data available")

### 4.4 Advanced Metrics Comparison

In [ ]:
# Compare key advanced metrics
metric_pairs = [
    ('baseline_entropy_mean', 'intervention_entropy_mean', 'Entropy'),
    ('baseline_js_divergence_mean', 'intervention_js_divergence_mean', 'JS Divergence'),
    ('baseline_macro_f1_mean', 'intervention_macro_f1_mean', 'Macro F1'),
    ('baseline_cohen_kappa_mean', 'intervention_cohen_kappa_mean', "Cohen's Kappa")
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (baseline_key, intervention_key, metric_name) in enumerate(metric_pairs):
    ax = axes[idx]
    
    demographics = []
    baseline_vals = []
    intervention_vals = []
    
    for demo, results in single_demo_results.items():
        metrics = results['overall_metrics']
        baseline_val = metrics.get(baseline_key)
        intervention_val = metrics.get(intervention_key)
        
        if baseline_val is not None and intervention_val is not None:
            demographics.append(demo.capitalize())
            baseline_vals.append(baseline_val)
            intervention_vals.append(intervention_val)
    
    if demographics:
        x = np.arange(len(demographics))
        width = 0.35
        
        ax.bar(x - width/2, baseline_vals, width, label='Baseline', alpha=0.8)
        ax.bar(x + width/2, intervention_vals, width, label='Intervention', alpha=0.8)
        
        ax.set_xlabel('Demographic')
        ax.set_ylabel(metric_name)
        ax.set_title(f'{metric_name}: Baseline vs Intervention')
        ax.set_xticks(x)
        ax.set_xticklabels(demographics, rotation=45, ha='right')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
    else:
        ax.text(0.5, 0.5, f'No {metric_name} data', 
                ha='center', va='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()

## 5. Per-Question Analysis

### 5.1 Extract Per-Question Results

In [ ]:
# Extract per-question results from fold_results
per_question_data = defaultdict(lambda: defaultdict(dict))

for demo, results in single_demo_results.items():
    for fold in results['fold_results']:
        for question, q_result in fold['test_results'].items():
            per_question_data[question][demo] = {
                'baseline_accuracy': q_result['baseline_accuracy'],
                'intervention_accuracy': q_result['intervention_accuracy'],
                'improvement': q_result['improvement'],
                'n_samples': q_result['n_samples']
            }

print(f"Found {len(per_question_data)} unique questions")
print(f"\nSample questions:")
for i, q in enumerate(list(per_question_data.keys())[:5]):
    print(f"  {i+1}. {q}")

### 5.2 Question-Level Improvement Heatmap

In [ ]:
# Create heatmap of improvements: Questions × Demographics
questions = list(per_question_data.keys())
demographics = list(single_demo_results.keys())

# Build improvement matrix
improvement_matrix = []
for question in questions:
    row = []
    for demo in demographics:
        if demo in per_question_data[question]:
            row.append(per_question_data[question][demo]['improvement'])
        else:
            row.append(np.nan)
    improvement_matrix.append(row)

improvement_matrix = np.array(improvement_matrix)

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, max(8, len(questions) * 0.3)))
im = ax.imshow(improvement_matrix, cmap='RdYlGn', aspect='auto', vmin=-20, vmax=20)

# Set ticks
ax.set_xticks(np.arange(len(demographics)))
ax.set_yticks(np.arange(len(questions)))
ax.set_xticklabels([d.capitalize() for d in demographics])
ax.set_yticklabels(questions)

# Rotate x labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Improvement (percentage points)', rotation=270, labelpad=20)

ax.set_title('Accuracy Improvement by Question and Demographic')
plt.tight_layout()
plt.show()

### 5.3 Interactive Question Explorer

In [ ]:
# Select a question to analyze in detail
question_to_analyze = list(per_question_data.keys())[0]  # Change index to explore different questions

print(f"Analyzing question: {question_to_analyze}")
print("\n" + "="*80)

# Show results for this question across demographics
question_results = []
for demo in demographics:
    if demo in per_question_data[question_to_analyze]:
        data = per_question_data[question_to_analyze][demo]
        question_results.append({
            'Demographic': demo.capitalize(),
            'Baseline Acc': f"{data['baseline_accuracy']*100:.1f}%",
            'Intervention Acc': f"{data['intervention_accuracy']*100:.1f}%",
            'Improvement': f"{data['improvement']:+.1f}",
            'Samples': data['n_samples']
        })

question_df = pd.DataFrame(question_results)
display(question_df)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
demo_names = [r['Demographic'] for r in question_results]
improvements = [float(r['Improvement']) for r in question_results]

colors = ['green' if imp > 0 else 'red' for imp in improvements]
ax.bar(demo_names, improvements, color=colors, alpha=0.7)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.set_xlabel('Demographic')
ax.set_ylabel('Improvement (percentage points)')
ax.set_title(f'Intervention Improvement for: {question_to_analyze}')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Per-Demographic Analysis

### 6.1 Which Demographics Improve Accuracy Most?

In [ ]:
# Rank demographics by overall improvement
demographic_rankings = []

for demo, results in single_demo_results.items():
    metrics = results['overall_metrics']
    demographic_rankings.append({
        'Demographic': demo.capitalize(),
        'Mean Improvement': metrics['improvement_mean'],
        'Std Improvement': metrics['improvement_std'] if metrics['improvement_std'] else 0,
        'Baseline Accuracy': metrics['baseline_accuracy_mean'] * 100,
        'Intervention Accuracy': metrics['intervention_accuracy_mean'] * 100
    })

ranking_df = pd.DataFrame(demographic_rankings).sort_values('Mean Improvement', ascending=False)

print("\n" + "="*80)
print("DEMOGRAPHIC RANKINGS (by Accuracy Improvement)")
print("="*80)
display(ranking_df)

print(f"\n🏆 Best performing demographic: {ranking_df.iloc[0]['Demographic']} (+{ranking_df.iloc[0]['Mean Improvement']:.2f} points)")
print(f"   Worst performing demographic: {ranking_df.iloc[-1]['Demographic']} ({ranking_df.iloc[-1]['Mean Improvement']:+.2f} points)")

### 6.2 Success Rate: Percentage of Questions with Positive Improvement

In [ ]:
# Calculate success rate per demographic
success_rates = []

for demo in demographics:
    positive_improvements = 0
    total_questions = 0
    
    for question, demo_results in per_question_data.items():
        if demo in demo_results:
            total_questions += 1
            if demo_results[demo]['improvement'] > 0:
                positive_improvements += 1
    
    if total_questions > 0:
        success_rate = (positive_improvements / total_questions) * 100
        success_rates.append({
            'Demographic': demo.capitalize(),
            'Questions with Improvement': positive_improvements,
            'Total Questions': total_questions,
            'Success Rate': f"{success_rate:.1f}%"
        })

success_df = pd.DataFrame(success_rates)
print("\n" + "="*80)
print("SUCCESS RATE BY DEMOGRAPHIC")
print("="*80)
display(success_df)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
demo_names = [r['Demographic'] for r in success_rates]
success_pcts = [float(r['Success Rate'].strip('%')) for r in success_rates]

bars = ax.bar(demo_names, success_pcts, alpha=0.7, color='steelblue')
ax.axhline(y=50, color='red', linestyle='--', linewidth=1, label='50% threshold')
ax.set_xlabel('Demographic')
ax.set_ylabel('Success Rate (%)')
ax.set_title('Percentage of Questions with Positive Improvement by Demographic')
ax.set_ylim(0, 100)
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bar, pct in zip(bars, success_pcts):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{pct:.1f}%',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

### 6.3 Effect Size Distribution

In [ ]:
# Plot distribution of improvements for each demographic
fig, ax = plt.subplots(figsize=(12, 6))

improvement_distributions = []
demo_labels = []

for demo in demographics:
    improvements = []
    for question, demo_results in per_question_data.items():
        if demo in demo_results:
            improvements.append(demo_results[demo]['improvement'])
    
    if improvements:
        improvement_distributions.append(improvements)
        demo_labels.append(demo.capitalize())

# Create violin plot
parts = ax.violinplot(improvement_distributions, positions=range(len(demo_labels)), 
                      showmeans=True, showmedians=True)

ax.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax.set_xticks(range(len(demo_labels)))
ax.set_xticklabels(demo_labels)
ax.set_xlabel('Demographic')
ax.set_ylabel('Improvement (percentage points)')
ax.set_title('Distribution of Accuracy Improvements Across Questions')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Summary Insights

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)

# Best demographic
best_demo = ranking_df.iloc[0]
print(f"\n1. BEST PERFORMING DEMOGRAPHIC")
print(f"   {best_demo['Demographic']}: +{best_demo['Mean Improvement']:.2f} points improvement")
print(f"   {best_demo['Baseline Accuracy']:.1f}% (baseline) → {best_demo['Intervention Accuracy']:.1f}% (intervention)")

# Success rates
if success_rates:
    best_success = max(success_rates, key=lambda x: float(x['Success Rate'].strip('%')))
    print(f"\n2. HIGHEST SUCCESS RATE")
    print(f"   {best_success['Demographic']}: {best_success['Success Rate']} of questions improved")
    print(f"   ({best_success['Questions with Improvement']}/{best_success['Total Questions']} questions)")

# Overall statistics
all_improvements = []
for demo in demographics:
    for question, demo_results in per_question_data.items():
        if demo in demo_results:
            all_improvements.append(demo_results[demo]['improvement'])

if all_improvements:
    print(f"\n3. OVERALL STATISTICS")
    print(f"   Mean improvement: {np.mean(all_improvements):+.2f} points")
    print(f"   Median improvement: {np.median(all_improvements):+.2f} points")
    print(f"   Std improvement: ±{np.std(all_improvements):.2f} points")
    print(f"   Positive improvements: {sum(1 for x in all_improvements if x > 0)}/{len(all_improvements)} ({100*sum(1 for x in all_improvements if x > 0)/len(all_improvements):.1f}%)")

# Intersectional comparison
if intersectional_results:
    for demographics, results in intersectional_results.items():
        metrics = results['overall_metrics']
        print(f"\n4. INTERSECTIONAL INTERVENTION ({demographics})")
        print(f"   Improvement: {metrics['improvement_mean']:+.1f} points")
        print(f"   {metrics['baseline_accuracy_mean']*100:.1f}% (baseline) → {metrics['intervention_accuracy_mean']*100:.1f}% (intervention)")
        
        # Compare to best single demographic
        if best_demo['Mean Improvement'] > metrics['improvement_mean']:
            print(f"   Note: Single demographic '{best_demo['Demographic']}' performed better (+{best_demo['Mean Improvement']:.2f} vs +{metrics['improvement_mean']:.2f})")
        else:
            print(f"   Note: Intersectional outperformed all single demographics!")

print("\n" + "="*80)

## 8. Export Results

Optionally export analysis results to CSV for further processing.

In [ ]:
# Uncomment to export
# summary_df.to_csv(f'analysis_summary_{RUN_ID}.csv', index=False)
# ranking_df.to_csv(f'demographic_rankings_{RUN_ID}.csv', index=False)
# print(f"Exported results to CSV files")